# Some preparation steps

1. Install `miniforge3` on your laptop in order to use Python and other packages.
2. Install python packages using `pip` as below.

In [1]:
# !curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
# !bash Miniforge3-$(uname)-$(uname -m).sh

In [2]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib beautifulsoup4 pyperclip openai scikit-learn

# Importing necessary functions

1. Make sure to import functions to properly display markdowns.
2. Import necessary functions we implemented from `retrieve_news_alert.py` and `summarize_with_openai`.

In [3]:
from IPython.display import Markdown, display

In [39]:
import numpy as np

In [4]:
from retrieve_news_alert import *
from summarize_with_openai import *

# Prepare API access

1. Prepare GMail authorization: if already done, it should happen without any prompt.
2. Read OpenAI API key from `openai_api_key.txt`.

In [5]:
service = authenticate_gmail()

In [6]:
with open('openai_api_key.txt', 'r') as f:
    key = f.readline().strip()

# Retrieve initial emails

1. Retrieve all emails from Bloomberg with "Alert" in the subject from Gmail.
2. Extract the main contents from HTML.
3. Create a topic list using OpenAI's API.

In [7]:
emails = get_emails(service, days=7, query=f'from:Bloomberg subject:Alert')

In [8]:
articles = format_emails(emails)

Total 97 news articles have been retrieved.


In [9]:
topics = extract_topics_from_titles(key, articles, instructions=[
                                   "Please sort them according to their prominence and cluster them as closely as possible.",
                                   "Please be as concise as possible.", 
                                   "Please provie only the cluster titles not the actual news titles.",
                                   "Please keep the bullet point list in one level.",
                                   "Please use markdown."
                               ])

In [10]:
display(Markdown(topics))

- US-China Relations and Taiwan Issues
- US Domestic Politics and Election
- Financial Regulation and Crypto Legislation
- Climate Talks and Environmental Regulation
- Employment and Union Issues
- Legislation regarding Privacy, Data and AI
- Biden Administration's Policies and Actions
- Global Diplomatic Relations and Conflicts
- US Trade Policies and Tariffs
- Healthcare and Abortion Legislation
- Big Tech and Antitrust Issues
- Trump Administration Activities

# Retrieve topic-relevant emails

1. Input the topic string based on the topic list above.
2. Retreive the emails with the new query from GMail.
3. Extract the main content from HTML.

In [31]:
topic = input("Please write the topic of your interest")

In [32]:
emails = get_emails(service, days=7, query=f'from:Bloomberg {topic}')

In [33]:
articles = format_emails(emails)

Total 6 news articles have been retrieved.


# Summarize individual articles

1. Summarize each article using OpenAI's GPT.
2. Print the summarized articles.

In [34]:
article_summaries = [
    {'title': article['title'], 'content': summarize_articles(key, [article], 
                                                              n_paragraphs=3, 
                                                              instructions=[
                                                                  "If possible, use bullet points to make it easily readable.",
                                                                  "Do not hesitate to paraphrase in order to make it readable and informative.",
                                                                  "Please use markdown."])} 
    for article in articles
]

In [37]:
summary_text = ""

for i, article in enumerate(article_summaries):
    summary_text += f"### {i+1} {article['title']}\n\n {article['content']}\n\n"

display(Markdown(
    f"## Selected Articles\n\n{summary_text}"
))

## Selected Articles

### 1 Kennedy Hits Trump’s Covid Response in Bid for Libertarian Vote

 * Robert F. Kennedy Jr. criticized former President Donald Trump's handling of the Covid-19 pandemic and his disregard for individual liberties and the Constitution in an appeal for votes from Libertarian Party members. Kennedy's remarks were made at the Libertarian National Committee convention in Washington where he indicated his view that there should be no exemption for pandemics in the U.S. Constitution.

* Kennedy's independent presidential campaign appears to be impacting the leading party candidates, namely Trump and President Joe Biden. Polling results place Kennedy's support near 10%, making him potentially the best performing third-party presidential candidate in three decades. His campaign is not only heightening competition but also causing a division among Libertarians, especially as the convention, unusually, invited non-party members to address its attendees. Trump also views the Libertarian Party, which typically sways votes away from Republican candidates, as key to growing his coalition.

* A former Democrat, Kennedy has strategically aligned himself with third parties to gather votes from a variety of political camps. Recognizing the potential of a diverse voter base, he gained ballot access in California with the help of the American Independent Party and in Michigan with the Natural Law Party. He has also won the endorsement of the Reform Party, which bolsters his ballot access in Florida. Kennedy's platform, which shows opposition to vaccine mandates, gun control, and abortion restrictions, and criticizes big tech companies, is likely to appeal to Libertarian voters.

### 2 Louisiana First in US to Make Possessing Abortion Pills a Crime

 - **New Abortion Pill Law in Louisiana:** Louisiana has become the first US State to criminalize the possession of the two primary abortion pills (mifepristone and misoprostol) without a prescription. The recent law treats these non-addictive medicines as controlled substances akin to certain narcotics with high potential for addiction or abuse. Republican Governor Jeff Landry, an opponent of abortion access, approved the bill into law, taking effect as of October 1.

- **Reaction on the New Legislation:** Critics of the law argue that it does not align with medical evidence. Mifepristone and misoprostol are shown to be safe and non-addictive according to studies from the US Food and Drug Administration and the Louisiana Society of Addictive Medicine. These are the most commonly used methods of terminating a pregnancy, accounting for 63% of abortions last year, as reported by the Guttmacher Institute. Julie F. Kay, executive director of abortion-rights advocacy group Abortion Coalition for Telemedicine, accuses lawmakers of attempting to save face ahead of elections.

- **Comparison of Risk Levels and Maternal Mortality Rate:** Contrary to popular belief, studies reveal that the risks of serious complications are lower in pill and surgical abortions than childbirth. A 2015 study in Obstetrics & Gynecology found that about 0.3% of people using mifepristone for abortion had a major complication, compared to 0.16% for surgical abortions during the first trimester and 1.4% for childbirth as per 2014 CDC estimates. Notably, states like Louisiana—ranking 47th out of 48 on maternal mortality rates from 2018 to 2021—enforce strict abortion laws. Mississippi, Georgia, and Alabama also have high maternal mortality rates and strict abortion legislation.

### 3 In Japan, the Bathroom Doubles as a Laundry Dryer: CityLab Daily

 - The bathroom found in contemporary Japanese homes doubles as a laundry dryer. The ceilings of these unique rooms are embedded with a combined blower and extractor fan, which blow warm air on damp clothes and effectively draw out the excess moisture from the room. An article found on CityLab lauds this innovation, stating that it is both more environmentally efficient and gentler on clothes than conventional tumble dryers.

- The World’s First Museum of Homelessness has recently opened in London; this timely establishment surfaces amid the UK's confronting housing crisis. It aims to emphasize that homelessness can potentially happen to anyone. On a different note, extensive research is being carried out to map the human ovary, cell by cell. The ultimate goal of this study is to create an artificial ovary that could potentially aid patients who have non-functioning ovaries, but want to reproduce. Lastly, industry leader IWG is betting on the significant prospect that hybrid workers outside the large cities are the future of the co-working industry.

- In other news, architectural salvage and reuse advocates across the US are making efforts to reform the $8.7 billion demolition industry in order to reduce carbon emissions and construction waste. These measures are being taken as the quality of new materials is deemed to be decreasing, and items are being discarded earlier than expected due to poor quality.

### 4 Kamala Harris Is Gaining Voters' Trust in Key States for Biden

 - Vice President Kamala Harris is gaining the trust of nearly half (48%) of swing-state voters, as suggested by a recent Bloomberg News/Morning Consult poll. This increasing trust in Harris stands to counteract GOP efforts to undermine Joe Biden’s fitness for office due to his age. Harris has built her reputation through several high-profile events that engage key Democratic voter groups, including a historic visit to a Minnesota abortion clinic and a powerful speech at a Civil Rights landmark in Selma, Alabama. Despite the increase in confidence, however, Harris trails Trump by 7% in a hypothetical head-to-head contest.

- Harris also stands out as the preferred successor among Democratic rising stars, should Biden be unable to continue in office. Her gaining popularity was evident during a Milwaukee, Wisconsin stop, where comedian D.L. Hughley publicly apologized for letting a media narrative shape his perception of Harris. Despite her initially uneven vice presidency, Harris has been proactively deployed by the White House to counteract political opponents, making appearances before audiences of color and young voters, communities critical for Democratic votes.
  
- In the event Biden drops out before November, 45% of swing-state voters support Harris, followed by Michigan Governor Gretchen Whitmer and Transportation Secretary Pete Buttigieg at 36%. Among potential Republican successors, if Trump is unable to continue, Florida Governor Ron DeSantis leads with 40%, and former UN Ambassador Nikki Haley comes in next at 38%. Harris's strategy includes private conversations with individuals outside her circle to shape her events and promote her “Economic Opportunity” tour, aimed at engaging Black men — a demographic targeted by Trump and the GOP.

### 5 After the Election, a Big Tax Fight Looms: Washington Edition

 - In an impending tax battle forecasted for 2025, the expiry of Trump-era individual tax cuts could directly impact Americans' wallets. Passed under the 2017 Tax Cuts and Jobs Act, the tax reductions for corporations were made permanent while those for individuals were temporary. As both parties have differing views on tax policies, passing more tax cuts may prove challenging without a "clean sweep" of the House, Senate, and the White House by one party, warns Bill Hoagland from the Bipartisan Policy Center. Inaction would result in increased income tax bills for most Americans.  

- On tax policy, Donald Trump plans to introduce tax cuts for businesses and individuals across all income levels. However, his advisors propose larger reductions than the ones in his previous law. Meanwhile, President Joe Biden intends to increase taxes on corporations, especially those operating offshore. He also plans to retain all Trump tax cuts for individuals earning up to $400,000. Interestingly, an overwhelming 77% of swing state voters support a billionaires tax to save Social Security, showed a Bloomberg News/Morning Consult poll. Regardless of the chosen plan, federal debt is likely to increase. 

- The news comes amidst other significant updates: the Supreme Court upheld a Republican-drawn congressional map in South Carolina; lawsuits alleging "entrenched" illegal monopoly were filed against Live Nation; new US home purchases declined due to high prices and mortgage rates; business activity in the U.S grew at a rapid pace in early May; Biden proposes to make Kenya the first African country to benefit from the Chips and Science Act. Biden is also nominating National Labor Relations Board Chair Lauren McFerran for a new five-year term. Finally, Biden plans to host Kenyan President Ruto for a state dinner at the White House.

### 6 Abortion Access on the Ballot in 2024 State Supreme Court Races

 * Abortion access is the central issue in the Georgia State Supreme Court election, the outcome of which may influence judicial candidates in 33 states. The election features former five-term congressman John Barrow challenging Justice Andrew Pinson, a Governor Brian Kemp appointee. While the campaign focuses heavily on personal liberties, it remains an exception as judicial contests in Georgia don't often get challenged. However, a win by Barrow could inspire future contenders to adopt similar campaign strategies.

* The significance of this race in Georgia comes in the wake of an expensive judicial contest in Wisconsin, where Janet Protasiewicz won a place in the state's Supreme Court. The outcome tipped the court's philosophical balance towards liberals. In Georgia, Barrow's potential victory would shift the balance in a nine-member panel, with seven justices appointed by Republican governors. The race is also pointing toward growing ties between judiciary elections and political divergences rather than focusing on legal expertise exclusively.

* The May timing of Georgia's election raises concerns about voter turnouts for these historically low-participation pollings. Despite the conservative majority in the state supreme court, a victory by Barrow would begin to alter the court's composition, according to the vice president of Reproductive Freedom For All. The nature of the election has also raised criticisms and debate over the politicization of such positions, revealing a shift from maintaining an impartial legal system to one increasingly influenced by partisan politics.



# Summarize all articles based on their respective summaries

1. Add the article numbers from above into `exclude_articles`, e.g., `exclude_articles=[1, 2]`, to exclude these irrelevant articles.
2. Use OpenAI API to summarize these articles.

In [56]:
exclude_articles = [3, 5]
included_articles = [i for i in range(len(articles)) if i+1 not in exclude_articles]
print("Included articles: ", ", ".join([str(i+1) for i in included_articles]))

Included articles:  1, 2, 4, 6


In [57]:
included_article_summaries = []

for i in included_articles:
    included_article_summaries.append(article_summaries[i])

In [58]:
summary = summarize_articles(key, 
                             included_article_summaries, 
                             n_paragraphs=5, 
                             instructions=[
                                 "If possible, use bullet points to make it easily readable.",
                                 "Please draw insights across multiple articles.",
                                 "Do not hesitate to paraphrase in order to make it readable and informative.",
                                 "Please use markdown."])

In [59]:
display(Markdown(summary))

- **Robert F. Kennedy Jr. Criticizes Trump and Bids for Libertarian Vote**: Robert F. Kennedy Jr.'s independent presidential campaign is garnering significant support, with polls indicating his voter base to be around 10%. His opposition to vaccine mandates, gun control, abortion restrictions and his criticism of big tech companies align with Libertarian voters' principles. This is causing a split among Libertarians and posing a threat to leading party candidates, including Trump and Biden.

- **Abortion Pills Criminalized in Louisiana**: Louisiana has become the first state in the U.S to make possession of abortion pills mifepristone and misoprostol without prescription a crime. The law has been denounced for its discordance with medical evidence, as studies by the FDA and the Louisiana Society of Addictive Medicine show the pills to be safe and non-addictive. The legislation has raised questions about the seriousness of complications amidst exceedingly high maternal mortality rates in states with strict abortion laws.

- **Kamala Harris Gains Voters' Trust in Key States**: Vice President Kamala Harris appears to be gaining trust among swing-state voters, with recent poll results reflecting 48% voter confidence in her. This shift could counteract GOP efforts to discredit Biden's fitness for office. Despite her growing popularity, Harris still trails Trump by 7% in a hypothetical head-to-head contest.

- **Abortion Central to 2024 State Supreme Court Races**: Abortion access has become a critical issue in the Georgia State Supreme Court election. The campaign, largely focused on personal liberties, hints at a shift from a judiciary centered on legal expertise to one increasingly influenced by party politics. This change could encourage future contenders to adopt similar campaign strategies.

- **Overall Insights**: Across all these news pieces, we see that prominent figures are gaining traction in the political arena as they address key issues that resonate with their target demographics. Abortion legislation, and individual liberties more widely, emerge as critical deliberation points both at the state and national levels. There's an also evident move towards political diversification with third-party candidates increasingly becoming a significant part of the electoral competition.